In [ ]:
from utils.load_env import load_env

envs = load_env()

# ComfyUI 服务地址
COMFYUI_URL = envs.get("COMFYUI_URL")

print(f"Using ComfyUI URL: {COMFYUI_URL}")

Environment variables loaded.
Using ComfyUI URL: http://home.chrissong.top:3818


In [ ]:
import requests
from io import BytesIO
from PIL import Image

# 加载工作流 api 然后执行

from utils.load_json import load_json

# WORKFLOW_API_FILE = "comfyui_wf_api/base_api.json"
WORKFLOW_API_FILE = "comfyui_wf_api/image2image.json"

workflow = load_json(WORKFLOW_API_FILE)


def queue_prompt(prompt):
    print("发送工作流到 ComfyUI 队列")
    response = requests.post(f"{COMFYUI_URL}/prompt", json={"prompt": prompt})
    return response.json()


def get_image(filename, subfolder, folder_type):
    print("从 ComfyUI 获取生成的图片")
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    response = requests.get(f"{COMFYUI_URL}/view", params=data)
    return Image.open(BytesIO(response.content))


# 使用示例
if __name__ == "__main__":
    # 创建工作流
    # prompt = create_text_to_image_prompt("An endless desert, fiery sun, blue sky")

    # 发送到 ComfyUI 处理
    response = queue_prompt(workflow)
    prompt_id = response["prompt_id"]
    print(f"Prompt queued with ID: {prompt_id}")

    # 获取生成的图像（实际使用时需要轮询等待完成）
    # 这里简化了流程，实际需要根据输出节点的信息获取图片
    # image = get_image("ComfyUI_00001_.png", "", "output")
    # image.show()

发送工作流到 ComfyUI 队列
Prompt queued with ID: 3ff152be-ebaa-4046-a17d-7c09787364c0
